# AS-AD model for the small open economy with fixed exchange rates
The AS-AD model for the small open economy with a credible fixed exchange rate can be summerized by the three equations: 
\begin{align}
\pi_t&=\pi_t^f+\gamma(y_t-\bar{y})+s_t \tag{AS} \\
\pi_t&=e_{t-1}^r+\pi_t^f-\frac{1}{\beta_1}(y_t-\bar{y}-z_t)  \tag{AD} \\
e_t^r&=e_{t-1}^r+\pi_t^f-\pi_t \tag{Exchange rate dynamics}
\end{align}

## User Guide
- Step 1: Click on the ***Runtime (Danish: Køreselstid)*** menu at the top and then ***Run all (Kør alle)***
- Step 2: Scroll down to the section ***Interactive plots*** to find the interactive plots*
- Step 3: You can now play around with the different sliders to investiage the effects of model parameters

## Importing packages

In [52]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import interact, interactive, fixed, interact_manual, Label, interactive_output
from IPython.display import display
import ipywidgets as widgets

## Plotting function

Defining functions that computes the AS and AD curve and IRFs

In [53]:
# Defining the AD curve
def AD(pif, erbef, beta1, ybar, z, grid):
    AD = erbef + pif - 1/beta1*(grid - ybar - z)
    return AD
    
# Defining the AS curve
def AS(pif, gamma, ybar, s, grid):
    AS = pif + gamma*(grid - ybar) + s
    return AS

# Computing the equilibrium in each period
def equilibrium(pif, erbef, beta1, gamma, ybar, z, s):
    y_eq = beta1/(1+gamma*beta1) * (erbef + 1/beta1*z-s) + ybar
    pi_eq = pif + beta1*gamma/(1+gamma*beta1) * (erbef + 1/beta1*z-s) + s
    return y_eq, pi_eq

# Coding a function that successively computes the model
def AS_solve(pif, beta1, gamma, ybar, z, s, N, grid):
    y_eq, pi_eq, er = np.zeros([N]), np.zeros([N]), np.zeros([N])
    AS_matrix, AD_matrix = np.zeros([len(grid), N]), np.zeros([len(grid), N])
    y_eq[0], pi_eq[0] = ybar, pif
    er[0] = 0
    AS_matrix[:,0], AD_matrix[:,0] = AS(pif, gamma, ybar, 0, grid), AD(pif, 0, beta1, ybar, 0, grid)
    for n in range(1,N):
            if n == 1:
                y_eq[n], pi_eq[n] = equilibrium(pif, er[n-1], beta1, gamma, ybar, z, s)
                er[n] = er[n-1] + pif - pi_eq[n]
                AS_matrix[:,n] = AS(pif, gamma, ybar, s, grid)
                AD_matrix[:,n] = AD(pif, er[n-1], beta1, ybar, z, grid)
            else:
                y_eq[n], pi_eq[n] = equilibrium(pif, er[n-1], beta1, gamma, ybar, 0, 0)
                er[n] = er[n-1] + pif - pi_eq[n]
                AS_matrix[:,n] = AS(pif, gamma, ybar, 0, grid)
                AD_matrix[:,n] = AD(pif, er[n-1], beta1, ybar, 0, grid)
    return y_eq, pi_eq, AS_matrix, AD_matrix

def IRF(pif, beta1, gamma, ybar, z, s, T):
    y_hat, pi_hat, er = np.zeros([T+1]), np.zeros([T+1]), np.zeros([T+1])
    y_hat[0], pi_hat[0], er[0] = ybar, pif, 0
    for t in range(1,T+1):
            if t == 1:
                y_hat[t], pi_hat[t] = equilibrium(pif, er[t-1], beta1, gamma, ybar, z, s)
                er[t] = er[t-1] + pif - pi_hat[t]
            else:
                y_hat[t], pi_hat[t] = equilibrium(pif, er[t-1], beta1, gamma, ybar, 0, 0)
                er[t] = er[t-1] + pif - pi_hat[t]             
    y_hat = y_hat - ybar
    pi_hat = pi_hat - pif
    return y_hat, pi_hat, er

Coding the plotting function for the interactive AS-AD diagram

In [60]:
# This function creates the interactive AS-AD diagram
def ASAD_inter_plot(pif, beta1, gamma, ybar, z, s, N, grid):
    y_seq, pi_seq, AS_seq, AD_seq = AS_solve(pif, beta1, gamma, ybar, z, s, N, grid)
    fig, ax1 = plt.subplots(figsize=(12, 6), dpi = 400)

    # Plot the curves
    ax1.scatter(y_seq, pi_seq, label='AD', color = 'black', zorder=4)
    ax1.plot(grid, AS_seq, label='AS', color = 'red')
    ax1.plot(grid, AD_seq, label='AD', color = 'blue')
    ax1.set_xlabel('Output (y)', fontsize = 12)
    ax1.set_ylabel('Inflation ($\pi$)', fontsize = 12)
    ax1.grid(True, zorder=0, linestyle = '--')
    ax1.set_ylim((0, int(max(pi_seq)+2)))
    ax1.set_xlim((grid[0], int(max(y_seq)+2)))
    x_coordinate = ybar
    plt.axvline(x=x_coordinate, color='r', linestyle='-', label='Vertical Line at x={}'.format(x_coordinate))
    
    # Adding annotation
    plt.text(ybar+0.05, int(max(pi_seq))+1.75, f'$LRAS$',fontsize = 14)
    plt.text(y_seq[0]+0.01, 0.05, r'$\bar{y}$',fontsize = 10)
    plt.text(0.01, pi_seq[0]+0.02, '$\pi^f$',fontsize = 10)
    for i in range(N):
        plt.annotate("",xy=(y_seq[i], pi_seq[i]), xytext=(y_seq[i], 0), arrowprops=dict(width=0.1, color='grey', headwidth=0.01, headlength=0.01))
        plt.annotate("",xy=(y_seq[i], pi_seq[i]), xytext=(0, pi_seq[i]), arrowprops=dict(width=0.1, color='grey', headwidth=0.01, headlength=0.01))
    for i in range(1,N):
        plt.text(y_seq[i]+0.01, 0.05, f'$y_{i}$',fontsize = 10)
        plt.text(0.01, pi_seq[i]+0.05, f'$\pi_{i}$',fontsize = 10)
        plt.text(y_seq[i]-0.03, pi_seq[i]+0.08, f'$E_{i}$',fontsize = 10)
    plt.title("AS-AD Diagram for Small Open Economy with Fixed Exchange Rate", fontsize = 16) 

    plt.show()

Coding the plotting function for the interactive impulse response diagram

In [61]:
# Defining the plotting function for the impulse response functions
def irf_inter_plot(pif, beta1, gamma, ybar, z, s, T):
    T_grid = [t for t in range(T+1)]
    y_hat, pi_hat, er = IRF(pif, beta1, gamma, ybar, z, s, T)
    fig, ax1 = plt.subplots(figsize=(12, 6), dpi = 400)

    # Plot on primary y-axis
    ax1.plot(T_grid, y_hat, label=r'Output $(\hat{y}_t = \equiv y_t - \bar{y})$', color = 'red')
    ax1.plot(T_grid, pi_hat, label=r'Inflation $(\hat{\pi}_t \equiv \pi_t - \pi^f)$', color = 'blue')
    ax1.plot(T_grid, er, label=r'Real Exchange Rate $(e^r)$', color = 'orange')
    ax1.set_xlabel('Horizon', fontsize = 12)
    ax1.set_ylabel('Deviation from steady state', fontsize = 12)
    ax1.set_xlim((T_grid[0], T_grid[-1]))
    ax1.grid(True, zorder=0, linestyle = '--')

    # Add legend
    lines, labels = ax1.get_legend_handles_labels()
    ax1.legend(lines, labels, loc='upper right')
    plt.title("IRFs for Open Economy AS-AD model with Fixed Exchange Rate", fontsize = 16) 

    plt.show() 

# Interactive plots

This cell just creates all the sliders needed for the plot to work (looks slightly involed but is very repetitive)

In [64]:
# Creating sliders for different model parameters
z_slider = widgets.FloatSlider(value=1,min=-4.0,max=4.0,step=0.1,description=r"$z$",continuous_update=False)
s_slider = widgets.FloatSlider(value=0,min=-4.0,max=4.0,step=0.1,description=r"$s$",continuous_update=False)
beta1_slider = widgets.FloatSlider(value=1.5,min=0.0,max=3.0,step=0.1,description=r"$\beta_1$",continuous_update=False)
gamma_slider = widgets.FloatSlider(value=1.5,min=0.0,max=3.0,step=0.1,description=r"$\gamma$",continuous_update=False)
ybar_slider = widgets.FloatSlider(value=2,min=0.0,max=4.0,step=0.5,description=r"$\bar{y}$",continuous_update=False)
pif_slider = widgets.FloatSlider(value=2,min=0.0,max=4.0,step=0.5,description=r"$\pi^f$",continuous_update=False)
N_slider = widgets.IntSlider(value=3,min=1,max=5,step=1,description=r"$N$",continuous_update=False)
T_slider = widgets.IntSlider(value=8,min=1,max=15,step=1,description=r"$T$",continuous_update=False)

# Arrange sliders in two columns using VBox and HBox
left_box = widgets.VBox([z_slider, s_slider, beta1_slider])
center_box = widgets.VBox([gamma_slider, ybar_slider, pif_slider])
right_box = widgets.VBox([N_slider])
hbox = widgets.HBox([left_box, center_box,right_box])

# Modify for the IRF plot
right_box_irf = widgets.VBox([T_slider])
hbox_irf = widgets.HBox([left_box, center_box, right_box_irf])

## AS-AD diagram

In [65]:
# Create the interactive plot
plot_output = interactive_output(ASAD_inter_plot, {'pif': pif_slider, 'beta1': beta1_slider, 'gamma': gamma_slider, 
                                              'ybar': ybar_slider, 'z': z_slider, 's': s_slider, 'N': N_slider, 
                                              'grid': fixed(np.linspace(0, 10, 100))})
display(hbox, plot_output)
plot_output.layout.height = '550px'

Output()

## Impulse response functions

In [59]:
# Create the interactive plot
plot_output = interactive_output(irf_inter_plot, {'pif': pif_slider, 'beta1': beta1_slider, 'gamma': gamma_slider, 
                                              'ybar': ybar_slider, 'z': z_slider, 's': s_slider, 'T': T_slider})
display(hbox_irf, plot_output)
plot_output.layout.height = '550px'

Output()